***required:***
1. list of sections with cable profile
2. BM due to permanent dead load
3. BM due to superimposed dead load
4. BM due to live load (max of all vehicle loads)

In [1]:
from Cross import Cross_section, section

[[3.8, 4.2], [3.8, 4.35], [3.8, 4.5], [3.8, 4.65], [3.8, 4.8], [6.8, 4.2], [6.8, 4.35], [6.8, 4.5], [6.8, 4.65], [6.8, 4.8], [4.4, 3.75], [5.0, 3.75], [5.6, 3.75], [6.2, 3.75]]


In [2]:
from bridge_specs import box
span = box['span']

In [3]:
end_cables_pos = [(0,1), (0,0), (1,0), (1,1)]
mid_cables_pos = [(0,0.25), (0,0), (1,0), (1,0.25)]
all_cable_pos = zip(end_cables_pos, mid_cables_pos)

### Cable profile

In [4]:
def cable_profile(end_cable_pos, mid_cable_pos, span_length, division=8):
    cable_profile = []
    x = [(j)/division*span_length for j in range(division+1)]
    for i in x:
        cable_pos_x = []
        for j in range(len(end_cable_pos)):
            h = end_cable_pos[j][1]-mid_cable_pos[j][1]
            a = end_cable_pos[j][1]
            y = a + 4*h*i/span_length - 4*h*i**2/span_length**2
            cable_pos_x.append((end_cable_pos[j][0], y))
        cable_profile.append(cable_pos_x)
    return cable_profile

### Moment of inertia including cable

In [5]:
# def MOI(cable_positions, cable_area, cable_moi, section_moi, centroid):
#     cable_moi_sum = 0
#     for i in cable_positions:
#         cable_moi_sum = cable_area*(i[1]-centroid[1])**2
#     moi = len(cable_positions)*cable_moi + section_moi + cable_moi_sum

### Eccentricity of cables

In [6]:
def eccentricity(cable_positions, centroid):
    cable_area_sum = len(cable_positions)
    cable_areaxpos_sum = 0
    for i in cable_positions:
        cable_areaxpos_sum+=(centroid[1]-i[1])
    return cable_areaxpos_sum/cable_area_sum

### Stress

In [7]:
list_of_sections = [section for i in range(9)] #dummy

prestressing_force_full = 2700 #dummy
prestressing_force_withloss = 0.7*prestressing_force_full

bm_dl = [1, 2, 3, 4, 5, 6, 7, 8, 9] #dl bata ayeko bm Dummy
bm_sidl = [1, 2, 3, 4, 5, 6, 7, 8, 9] #sidl bata ayeko bm Dummy
bm_ll = [1, 2, 3, 4, 5, 6, 7, 8, 9] #ll bata ayeko bm Dummy

In [8]:
def tau(section, bm):
    tau_top = bm*section.ymax/section.I[0]
    tau_bot = bm*section.ymin/section.I[0]
    return (tau_top, tau_bot)

**Case 1: Rare combination**

presetressing stress

In [9]:
# bm_pre = [prestressing_force_full*eccentricity(i.cable_positions, i.Centroid[1]) for i in list_of_sections]
bm_pre = [1, 2, 3, 4, 5, 6, 7, 8, 9] #dummy

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))]

# for index, i in enumerate(list_of_sections):
#     list_tau_pre.append(tau(i, bm_pre[index]))
    
list_tau_ax = [prestressing_force_full/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

stress due to ded load

In [10]:
list_tau_DL = [tau(list_of_sections[index], bm_dl[index]) for index in range(len(list_of_sections))]

resultant

In [11]:
list_tau_resultant = [(list_tau_preTotal[index][0] + list_tau_DL[index][0], list_tau_pre[index][1] + list_tau_DL[index][1]) for index in range(len(list_of_sections))]

check (*left blank for now*)

**Case 2: Quasi Permanent Combination**

presetressing stress

In [12]:
# bm_pre = [prestressing_force_withloss*eccentricity(i.cable_positions, i.Centroid[1]) for i in list_of_sections]
bm_pre = [1, 2, 3, 4, 5, 6, 7, 8, 9] #dummy

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))]
  
list_tau_ax = [prestressing_force_withloss/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

stress due to super imposed ded load

In [13]:
list_tau_SIDL = [tau(list_of_sections[index], bm_sidl[index]) for index in range(len(list_of_sections))]

stress due to live load

In [14]:
list_tau_LL = [tau(list_of_sections[index], bm_ll[index]) for index in range(len(list_of_sections))]

resultant

In [15]:
list_tau_resultant = [(list_tau_preTotal[index][0] + list_tau_DL[index][0]+ list_tau_SIDL[index][0]+ list_tau_LL[index][0], list_tau_preTotal[index][1] + list_tau_DL[index][1]+ list_tau_SIDL[index][1]+ list_tau_LL[index][1]) for index in range(len(list_of_sections))]

check (*left blank for now*)